# Gaussian Bayesian Classifier

### Mastery Checkpoint 1  
CSC 466, Winter 2022  
Samay Nathani

In [1]:
from pathlib import Path
home = str(Path.home())

In [2]:
%load_ext autoreload
%autoreload 2

import Lab2_helper

Import our datasets

In [8]:
import pandas as pd
import numpy as np
titanic_df = pd.read_csv(
    f"{home}/csc-466-student/data/titanic.csv"
)
features = ['Pclass','Survived','Sex','Age']
titanic_df = titanic_df.loc[:,features]
display(titanic_df.head())
titanic_df.loc[:,'Pclass']=titanic_df['Pclass'].fillna(titanic_df['Pclass'].mode()).astype(int)
titanic_df.loc[:,'Age']=titanic_df['Age'].fillna(titanic_df['Age'].median())
titanic_df.loc[:,'Age']=(titanic_df['Age']/10).astype(str).str[0].astype(int)*10
titranic_df = titanic_df.dropna()
X = titanic_df.drop("Survived",axis=1)
y = titanic_df["Survived"]
probs = Lab2_helper.class_conditional(X,y)
priors = Lab2_helper.compute_priors(y)
x = titanic_df.drop("Survived",axis=1).loc[2]
post_probs = Lab2_helper.posteriors(probs,priors,x)

,Pclass,Survived,Sex,Age
0,3,0,male,22.0
1,1,1,female,38.0
2,3,1,female,26.0
3,1,1,female,35.0
4,3,0,male,35.0


Defining the Gaussian Bayesian Classifier

In [9]:
class GaussianBayes:
    
    def compute_priors(y):
        value_counts = y.value_counts().sort_index()
        total_values = len(y)
        indexes = [y.name+"="+str(v) for v in value_counts.index.tolist()]
        priors = dict(zip(indexes, value_counts / total_values))
        return priors
    
    def compute_normal_distribution(mean, stddev, x):
        exp = -1*(x-mean)**2 / 2*(stddev**2)
        base = math.e / (stddev) ** math.sqrt(2 * math.pi)
        return base ** exp
    
    
    # Since we are assuming a normal distribution, this is where we use the mean, std, and variance to calculate the class conditional probability. 
    def specific_class_conditional(x,xv,y,yv):
        # todo: replace binning with normal distribution calculations
        likelihoods = {}
        priors = {}
        
        for ux in x.unique():
            y_vals = y.loc[x==ux]
            print(y_vals)
            binned, bins = pd.cut(y.loc[x==ux], 5, retbins=True)
            binned_counts = binned.value_counts()
            bin_value = pd.cut([yv], bins)[0]
            
            if bin_value in binned_counts.index:
                count = binned_counts.loc[bin_value]
            else:
                count = 0
            
            likelihoods[ux] = count / sum(binned_counts)
            #priors[ux] = sum(binned_counts) / len(x)

        denom = 0
        
        #for k in priors.keys():
        #    denom += (priors[k]*likelihoods[k])

        #classcond = likelihoods[xv]*priors[xv] / denom
        #return classcond


    def class_conditional(X,y):
        probs = {}
        
        for eachy in y.unique():
            for col in X.columns:
                for eachx in X[col].unique():
                    probs[col + "=" + str(eachx) + "|" + y.name + "=" + str(eachy)] = specific_class_conditional(X[col], eachx, y, eachy)
        return probs

    def posteriors(probs,priors,x):
        post_probs = {}
        denom = 0
        for k in priors.keys():
            numerator = 1
            postkey = ""
            
            for idx in x.index:
                postkey += idx + "=" + str(x[idx]) + ","
                probkey = idx + "=" + str(x[idx]) + "|" + k
                
                if probkey not in probs:
                    numerator *= 0
                else:
                    numerator *= probs[probkey]
            
            numerator*=priors[k]
            post_probs[k + "|" + postkey[:-1]] = numerator
            denom+=numerator
            
        for k, v in post_probs.items():
            if denom != 0:
                post_probs[k] = v / denom
            else:
                post_probs[k] = 1 / len(list(priors.keys()))
        
        return post_probs


In [10]:
gb = GaussianBayes()